In [ ]:
# @title ☁️ WebDAV ⇄ ♻️ GDrive Sync (Rclone)
# @markdown ### ☁️  Credentials
WEBDAV_HOST = "" # @param {type:"string"}
WEBDAV_USER = "" # @param {type:"string"}
WEBDAV_PASS = "" # @param {type:"string"}
# @markdown ### ♻️ Google Drive Destination
DRIVE_FOLDER_NAME = "webdav_backup" # @param {type:"string"}

import os
import subprocess
import urllib.parse
import re
import threading
import ipywidgets as widgets
from google.colab import drive
from IPython.display import display, clear_output, HTML

# --- 1. SETUP ---
def setup_environment():
    if not os.path.exists("/usr/bin/rclone"):
        subprocess.run("sudo -v ; curl https://rclone.org/install.sh | sudo bash", shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    try:
        from webdav3.client import Client
    except ImportError:
        subprocess.run("pip install webdavclient3", shell=True, check=True, stdout=subprocess.DEVNULL)

    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

setup_environment()
from webdav3.client import Client

# --- 2. CONFIG ---
def get_rclone_config():
    try:
        obscured = subprocess.check_output(["rclone", "obscure", WEBDAV_PASS], text=True).strip()
    except:
        obscured = WEBDAV_PASS

    return f"""
[pikpak]
type = webdav
url = {WEBDAV_HOST}
vendor = other
user = {WEBDAV_USER}
pass = {obscured}
"""

with open("rclone.conf", "w") as f:
    f.write(get_rclone_config())

def skip_check(self, remote_path): return True
Client.check = skip_check

options = {
    'webdav_hostname': WEBDAV_HOST,
    'webdav_login': WEBDAV_USER,
    'webdav_password': WEBDAV_PASS,
    'disable_check_exists': True,
    'timeout': 60
}
client = Client(options)

# --- 3. VERIFICATION & RUN ---

def human_readable(bytes_size):
    if bytes_size is None: return "0 B"
    try:
        bytes_size = float(bytes_size)
    except:
        return "-"

    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024.0:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024.0
    return f"{bytes_size:.2f} PB"

def get_size_rclone(path):
    try:
        # Internally runs rclone size to get accurate folder/file size
        cmd = ["rclone", "size", path, "--config", "rclone.conf", "--json"]
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        import json
        data = json.loads(result.stdout)
        return data.get("bytes", 0), data.get("count", 0)
    except:
        return 0, 0

# --- 4. GUI LOGIC ---

# CSS for Dark Mode
dark_css = widgets.HTML("""
<style>
    /* Main Container */
    .widget-area {
        background-color: #121212 !important;
        color: #e0e0e0;
        padding: 15px;
        border-radius: 8px;
        border: 1px solid #333;
    }

    /* Input Fields & Dropdowns */
    .widget-text input, .widget-dropdown select {
        background-color: #2d2d2d !important;
        color: #fff !important;
        border: 1px solid #555 !important;
    }
    .widget-text label { color: #ccc !important; font-weight: bold; }

    /* Standard Buttons (Back, etc) */
    .widget-button {
        color: #fff !important;
        background-color: #333 !important;
        border: 1px solid #555;
        font-family: monospace;
    }
    .widget-button:hover { background-color: #444 !important; }

    /* File/Folder Buttons (Blue) */
    .widget-button.mod-info {
        background-color: #0d47a1 !important; /* Dark Blue */
        color: white !important;
        border-color: #002171;
        text-align: left;
    }

    /* Select/Action Buttons (Orange/Gold) */
    .widget-button.mod-warning {
        background-color: #f57f17 !important; /* Dark Orange */
        color: black !important;
        font-weight: bold;
    }

    /* Success/Download Buttons (Green) */
    .widget-button.mod-success {
        background-color: #1b5e20 !important; /* Dark Green */
        color: white !important;
    }

    /* Output Area */
    .output_subarea {
        background-color: #000 !important;
        color: #00ff00 !important;
        border: 1px solid #333;
    }
    pre { color: #e0e0e0 !important; }
</style>
""")

out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='10px', height='300px', overflow='scroll'))
path_display = widgets.Text(value='/', description='Path:', disabled=True, layout=widgets.Layout(width='98%'))
btn_back = widgets.Button(description='.. (Up)', icon='arrow-up', layout=widgets.Layout(width='100px'))
file_list_box = widgets.VBox()
current_path = "/"

def run_rclone_job(action_type, remote_path, item_name):
    """Executes Rclone"""
    with out:
        clear_output()
        print(f"🚀 RCLONE STARTED: {action_type}")

        source_full = f"pikpak:{remote_path}"
        dest_path = f"/content/drive/MyDrive/{DRIVE_FOLDER_NAME}/{item_name}"

        print(f"   Source: {source_full}")
        print(f"   Dest:   {dest_path}")
        print("="*60)

        cmd = ["rclone", "copy" if action_type == "Folder" else "copyto"]
        cmd.append(source_full)
        cmd.append(dest_path)
        cmd.extend([
            "--config", "rclone.conf",
            "--transfers", "8",
            "--checkers", "16",
            "--progress",
            "--stats", "1s",
            "--user-agent", "Mozilla/5.0",
            "--ignore-existing"
        ])

        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)
        for line in process.stdout:
            print(line, end='')
        process.wait()

        if process.returncode != 0:
            print("\n❌ ERROR: Transfer failed.")
            return
        else:
            print("\n✅ TRANSFER COMPLETE!")

        print("\n🔍 VERIFYING INTEGRITY...")
        src_bytes, src_count = get_size_rclone(source_full)
        dst_bytes, dst_count = get_size_rclone(dest_path)

        print("-" * 40)
        print(f"☁️ WebDAV Cloud:  {human_readable(src_bytes)} ({src_count} files)")
        print(f"♻️ GDrive Saved:  {human_readable(dst_bytes)} ({dst_count} files)")
        print("-" * 40)

        # --- SUMMARY VERIFICATION ---
        if src_bytes == dst_bytes:
            print("✅ INTEGRITY MATCH: Summary sizes match.")
        else:
            print(f"⚠️ SUMMARY MISMATCH: Counts differ.")

        # --- DEEP VERIFICATION ---
        print("\n🕵️ DEEP VERIFICATION: Checking if files exist on GDrive with matching sizes...")
        print("   (This may take time, scanning destination...)")

        check_cmd = [
            "rclone", "check", source_full, dest_path,
            "--config", "rclone.conf",
            "--size-only",
            "--one-way",
            "--quiet"
        ]

        check_proc = subprocess.run(check_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        if check_proc.returncode == 0:
            print("✅ VERIFIED AVAILABLE: All files from ⚡WebDAV exist on ♻️GDrive.")
        else:
            print("❌ VERIFICATION FAILED: Some files are missing or size mismatched.")
            print("   You may need to run the transfer again.")

        print("\n👉 Scroll up and click '.. (Up)' to continue browsing.")

def on_click_item(b):
    global current_path
    item = b.tag
    if item['isdir']:
        current_path = item['path']
        refresh_browser()
    else:
        with out:
            clear_output()
            print(f"📄 File: {item['name']}")
            print(f"💾 Size: {item['display_size']}")
            btn_dl = widgets.Button(description="Download File", button_style='success', icon='download')
            def do_dl(b): run_rclone_job("File", item['path'], item['name'])
            btn_dl.on_click(do_dl)
            display(btn_dl)

def on_click_folder_dl(b):
    item = b.tag
    with out:
        clear_output()
        print(f"📂 Folder: {item['name']}")
        btn_dl = widgets.Button(description="Start Transfer", button_style='success', icon='rocket')
        def do_dl(b): run_rclone_job("Folder", item['path'], item['name'])
        btn_dl.on_click(do_dl)
        display(btn_dl)

def on_go_back(b):
    global current_path
    if current_path != '/':
        current_path = os.path.dirname(current_path.rstrip('/'))
        if not current_path: current_path = '/'
        refresh_browser()

btn_back.on_click(on_go_back)

def refresh_browser():
    global current_path
    path_display.value = urllib.parse.unquote(current_path)
    with out:
        clear_output()
        print("⏳ Fetching file list from WebDAV...")

    try:
        items = client.list(current_path, get_info=True)
        items = [i for i in items if urllib.parse.unquote(i['path']).rstrip('/') != urllib.parse.unquote(current_path).rstrip('/')]
        items.sort(key=lambda x: (not x['isdir'], x['name']))

        # Prepare list for rendering
        processed_buttons = []

        # Count dirs for progress bar
        dirs_to_process = [i for i in items if i['isdir']]
        total_dirs = len(dirs_to_process)
        current_dir_count = 0

        for i in items:
            name = urllib.parse.unquote(i['name'])
            path = urllib.parse.unquote(i['path'])
            is_dir = i['isdir']

            display_size = ""

            if is_dir:
                current_dir_count += 1
                # --- PROGRESS UPDATE ---
                with out:
                    clear_output(wait=True)
                    print(f"⏳ Calculating sizes... ({current_dir_count}/{total_dirs})")
                    print(f"📂 Scanning: {name}")

                # Blocking Calculation
                try:
                    full_remote_path = f"pikpak:{path}"
                    f_bytes, f_count = get_size_rclone(full_remote_path)
                    formatted_size = human_readable(f_bytes)
                    display_size = f"({formatted_size})"
                except:
                    display_size = "(-)"

                icon = "📁"
                style = 'info'
            else:
                # Files usually have size in metadata, no need to scan
                raw_size = i.get('size') or i.get('content_length') or i.get('contentLength')
                formatted_size = human_readable(raw_size)
                display_size = f"({formatted_size})"
                icon = "📄"
                style = ''

            # Create Button
            btn_label = f"{icon} {display_size} {name}"
            btn_item = widgets.Button(description=btn_label, layout=widgets.Layout(width='75%', text_align='left'), button_style=style)
            btn_item.tag = {'path': path, 'name': name, 'isdir': is_dir, 'display_size': display_size}
            btn_item.on_click(on_click_item)

            if is_dir:
                btn_dl = widgets.Button(description="Select", button_style='warning', layout=widgets.Layout(width='80px'))
                btn_dl.tag = {'path': path, 'name': name}
                btn_dl.on_click(on_click_folder_dl)
                processed_buttons.append(widgets.HBox([btn_item, btn_dl]))
            else:
                processed_buttons.append(widgets.HBox([btn_item]))

        # Render all at once
        file_list_box.children = tuple(processed_buttons)
        with out: clear_output()

    except Exception as e:
        with out: print(f"❌ Error: {e}")

# Construct the Dark UI Wrapper
ui = widgets.VBox([
    dark_css,
    widgets.HTML("<h2>WebDAV Explorer (Dark Mode)</h2>"),
    widgets.HBox([btn_back, path_display]),
    widgets.HTML("<hr style='border-color: #555;'>"),
    file_list_box,
    out
])
ui.add_class('widget-area')

display(ui)
refresh_browser()

# ☁️ WebDAV ⇄ ♻️ GDrive Sync (Rclone)

## 🚠 Quick Start Guide

### Step 1: Add Your WebDAV Credentials to Colab Secrets
1. Click the **🔑 Secrets** icon on the left sidebar
2. Click **"+Add new secret"** and create TWO secrets:
   - **Name:** `WEBDAV_USER` → **Value:** Your WebDAV username
   - **Name:** `WEBDAV_PASS` → **Value:** Your WebDAV password
5. ### Configure Your WebDAV Server
**WEBDAV_HOST** = "your-webdav-server-url"

Examples:
- PikPak: `https://dav.mypikpak.com`
- Nextcloud: `https://nextcloud.example.com/remote.php/dav/`
- OwnCloud: `https://owncloud.example.com/remote.php/dav/`
- Seafile: `https://seafile.example.com/`
- Yandex.Disk: `https://webdav.yandex.com/`

Then proceed to Step 2.

4. Grant notebook access when prompted

### Step 2: Run the Code Cell Below

Click the play button (▶) to execute the main code. The notebook will:
- Install Rclone and WebDAVClient3
- Mount your Google Drive
- Launch the WebDAV browser UI

### Step 3: Use the Browser

| Action | Button | Color |
|--------|--------|-------|
| Open folder | Click folder name | 🔵 Blue |
| Select folder to transfer | "Select" button | 🟠 Orange |
| Go back up | ".. (Up)" button | 🟠 Orange |
| Download file | "Download File" | 🟢 Green |
| Start transfer | "Start Transfer" | 🟢 Green |

### What Happens During Transfer

- 🚀 **Transfer starts**: Rclone begins syncing files
- 🔍 **Verification**: Automatic integrity check
- ✅ **Complete**: Files verified and saved to GDrive

### ⚠️ Important Notes

- **NO credentials stored** in code (uses Colab Secrets)
- **Safe to share** on GitHub - credentials never exposed
- Transfer speed depends on **internet bandwidth**
- Verify transfers complete before deleting sources